In [5]:
import cx_Oracle
import pandas as pd

f=open("pwds.txt","r")
lines=f.readlines()
usr=lines[0]
pw=lines[1]
f.close()
path = 'C:\ORACLE\instantclient_19_11'

host = 'ALASTRIA'
port = 1524
service = 'CW1T2'
#cx_Oracle.init_oracle_client(lib_dir= path)

cx_Oracle.clientversion()  
dsn_tns = cx_Oracle.makedsn(host, port, service_name=service)

conn = cx_Oracle.connect(user=usr, password=pw, dsn=dsn_tns)

c = conn.cursor()

no_sus = """

select "WBC_CASE_D"."CASE_NUM" "Case_Number", "WBC_DATE_D_Start"."CALENDAR_DATE" "Start_Date", "WBC_DATE_D_Exit"."CALENDAR_DATE" "Exit_Date", avg("coguda30"."WEEKS_TO_ACHV_SUST_EMP_ELAPSED") "c4", "WBC_ES_STAGE_OF_SERVICE_D"."STAGE_OF_SERVICE_NAME" "Stage_of_Employment_Service1"
 from "CDW"."WBC_CASE_D" "WBC_CASE_D", "CDW"."WBC_DATE_D" "WBC_DATE_D_Start", "CDW"."WBC_DATE_D" "WBC_DATE_D_Exit", "CDW"."WBC_CASE_F" "coguda30", "CDW"."WBC_OFFICE_D" "coguda31", "CDW"."WBC_ES_STAGE_OF_SERVICE_D" "WBC_ES_STAGE_OF_SERVICE_D"
 where  "coguda30"."OFFICE_ORG_WID"="coguda31"."OFFICE_ORG_WID" and "WBC_DATE_D_Start"."FISCAL_YEAR">2019 and "coguda30"."ES_SRV_PLN_TYPE" in ('Employment Services') and "WBC_ES_STAGE_OF_SERVICE_D"."STAGE_OF_SERVICE_NAME" in ('Before Sustainable Employment Achieved', 'Left Before Outcome', 'Not Available') and "coguda30"."CASE_TYPE_WID"=1 and "coguda30"."SNAPSHOT_DT_WID"=(
select snapshot_dt_wid
 from cdw.wbc_run_ctl_d) and "coguda30"."CASE_WID"="WBC_CASE_D"."CASE_WID" and "coguda30"."ES_STAGE_OF_SERVICE_WID"="WBC_ES_STAGE_OF_SERVICE_D"."ES_STAGE_OF_SERVICE_WID" and "WBC_DATE_D_Start"."DATE_WID"="coguda30"."PLAN_START_DT_WID" and "coguda30"."PLAN_CLOSED_DT_WID"="WBC_DATE_D_Exit"."DATE_WID"
 group by "WBC_CASE_D"."CASE_NUM", "WBC_DATE_D_Exit"."CALENDAR_DATE", "WBC_DATE_D_Start"."CALENDAR_DATE", "WBC_ES_STAGE_OF_SERVICE_D"."STAGE_OF_SERVICE_NAME"


"""

sus = """

select "WBC_CASE_D"."CASE_NUM" "Case_Number", "WBC_DATE_D_Start"."CALENDAR_DATE" "Start_Date", "WBC_DATE_D_Exit"."CALENDAR_DATE" "Exit_Date", avg("coguda30"."WEEKS_TO_ACHV_SUST_EMP_ELAPSED") "c4", "WBC_ES_STAGE_OF_SERVICE_D"."STAGE_OF_SERVICE_NAME" "Stage_of_Employment_Service1"
 from "CDW"."WBC_CASE_D" "WBC_CASE_D", "CDW"."WBC_DATE_D" "WBC_DATE_D_Start", "CDW"."WBC_DATE_D" "WBC_DATE_D_Exit", "CDW"."WBC_CASE_F" "coguda30", "CDW"."WBC_OFFICE_D" "coguda31", "CDW"."WBC_ES_STAGE_OF_SERVICE_D" "WBC_ES_STAGE_OF_SERVICE_D"
 where "coguda30"."OFFICE_ORG_WID"="coguda31"."OFFICE_ORG_WID" and "WBC_DATE_D_Start"."FISCAL_YEAR">2019 and "coguda30"."ES_SRV_PLN_TYPE" in ('Employment Services') and "WBC_ES_STAGE_OF_SERVICE_D"."STAGE_OF_SERVICE_NAME" in ('Left Before Completing Sustainment', 'Left After Completing Sustainment', 'In Sustainment') and "coguda30"."CASE_TYPE_WID"=1 and "coguda30"."SNAPSHOT_DT_WID"=(
select snapshot_dt_wid
 from cdw.wbc_run_ctl_d) and "coguda30"."CASE_WID"="WBC_CASE_D"."CASE_WID" and "coguda30"."ES_STAGE_OF_SERVICE_WID"="WBC_ES_STAGE_OF_SERVICE_D"."ES_STAGE_OF_SERVICE_WID" and "WBC_DATE_D_Start"."DATE_WID"="coguda30"."PLAN_START_DT_WID" and "coguda30"."PLAN_CLOSED_DT_WID"="WBC_DATE_D_Exit"."DATE_WID"
 group by "WBC_CASE_D"."CASE_NUM", "WBC_DATE_D_Exit"."CALENDAR_DATE", "WBC_DATE_D_Start"."CALENDAR_DATE", "WBC_ES_STAGE_OF_SERVICE_D"."STAGE_OF_SERVICE_NAME"




"""
df = pd.read_sql(sus, con=conn)
print(df.shape)
df2 = pd.read_sql(no_sus, con=conn)
print(df2.shape)


(25277, 5)
(37029, 5)


In [8]:
import datetime
year_ago = datetime.datetime.now() - datetime.timedelta(days=365)
print(df2.head)
df2['Start_Date'] = pd.to_datetime(df2['Start_Date'], format = '%Y-%m-%d')
print(df2.dtypes)
df3 = df2.loc[(df2['Stage_of_Employment_Service1'] == 'Before Sustainable Employment Achieved') &  (df2['Start_Date']< year_ago)]
print(df2['Stage_of_Employment_Service1'].value_counts().to_frame())
df4 = df2[df2['Stage_of_Employment_Service1'] == 'Left Before Outcome']
combined_no_sus = df4.append(df3)
print(combined_no_sus.shape)

<bound method NDFrame.head of            Case_Number Start_Date  Exit_Date    c4  \
0      1-421153-103867 2019-11-19 2021-07-14  None   
1      1-421153-105307 2020-01-23 2020-11-05  None   
2      1-421153-141264 2020-11-24 2021-08-05  None   
3      1-421153-141559 2021-03-15 2021-06-07  None   
4      1-421153-142605 2020-01-15 2021-08-30  None   
...                ...        ...        ...   ...   
37024    1-65439190970 2021-10-04        NaT  None   
37025    1-65439204536 2021-10-04        NaT  None   
37026    1-65439276405 2021-10-04        NaT  None   
37027    1-65439714237 2021-10-04        NaT  None   
37028    1-65461662707 2021-10-05        NaT  None   

                 Stage_of_Employment_Service1  
0                         Left Before Outcome  
1                         Left Before Outcome  
2                         Left Before Outcome  
3                         Left Before Outcome  
4                         Left Before Outcome  
...                              

In [9]:
combined_no_sus['sus'] = 0
df['sus'] = 1
df_combo = combined_no_sus.append(df)
print(df.shape)
print(combined_no_sus.shape)
print(df_combo.shape)

(25277, 6)
(26585, 6)
(51862, 6)


In [7]:
q = """select distinct "WBC_OES_APPLICATION_D"."EXPLAINING_TO_OTHERS" "Explaining_to_Others", "WBC_OES_APPLICATION_D"."GET_ALONG_WITH_OTHERS" "Getting_Along_with_Others", "WBC_OES_APPLICATION_D"."LEARNING_NEW_THINGS" "New_Things", "WBC_OES_APPLICATION_D"."ORGANIZING_INFORMATION" "Organizing", "WBC_OES_APPLICATION_D"."PROBLEM_SOLVING" "Problem_Solving", "WBC_OES_APPLICATION_D"."READING_INFORMATION" "Reading_and_Understanding", "WBC_OES_APPLICATION_D"."USING_COMPUTERS" "Using_Computers", "WBC_OES_APPLICATION_D"."CREATING_DOCUMENTS" "Working_with_Documents", "WBC_OES_APPLICATION_D"."WORKING_WITH_MATH" "Working_with_Math", "WBC_OES_APPLICATION_D"."WRITING_INFORMATION" "Writing", "WBC_OES_APPLICATION_D"."MOST_RECENT_WORK_EMPLOYER" "Employer_Name", "WBC_OES_APPLICATION_D"."MOST_RECENT_WORK_END_DATE" "End_Date", "WBC_OES_APPLICATION_D"."MOST_RECENT_WORK_INDUSTRY" "Industry", "WBC_OES_APPLICATION_D"."REASON_FOR_LEAVING_EMPLOYMENT" "Reason_for_Leaving", "WBC_OES_APPLICATION_D"."MOST_RECENT_WORK_START_DATE" "Start_Date", "WBC_OES_APPLICATION_D"."MOST_RECENT_WORK_JOB_TITLE" "Work_Title", "WBC_OES_APPLICATION_D"."CURRENTLY_IN_SCHOOL" "Currently_In_School", "WBC_OES_APPLICATION_D"."CURRENTLY_IN_SCHOOL_STATUS" "c18", "WBC_OES_APPLICATION_D"."AREA_OF_STUDY" "Education_Area_of_Study", "WBC_OES_APPLICATION_D"."INDIGENOUS_TYPE" "Indigenous_Type", "WBC_OES_APPLICATION_D"."STATUS_INDIAN" "Status_Indian", "WBC_OES_APPLICATION_D"."IMMIGRATION_DATE" "Immigration_Date", "WBC_OES_APPLICATION_D"."REFUGEE_TYPE" "Refugee_Type", "WBC_OES_APPLICATION_D"."CITIZENSHIP_STATUS" "Citizenship_Status", "WBC_OES_APPLICATION_D"."GENDER" "Gender", "WBC_OES_APPLICATION_D"."CELL_PHONE" "Cell_Phone", "WBC_OES_APPLICATION_D"."WORK_PHONE" "Work_Phone", "WBC_OES_APPLICATION_D"."HOME_PHONE" "Home_Phone", "WBC_OES_APPLICATION_D"."ALTERNATE_NUMBER" "Alternate_Phone_Number", "WBC_OES_APPLICATION_D"."FRANCOPHONE" "Francophone", "WBC_OES_APPLICATION_D"."RECENT_IMMIGRANT" "Recent_Immigrant", "WBC_OES_APPLICATION_D"."INDIGENOUS_PERSON" "Indigenous_Person", "WBC_OES_APPLICATION_D"."PERSON_WITH_DISABILITY" "Person_with_a_Disability", "WBC_OES_APPLICATION_D"."SURVIVOR_OF_VIOLENCE" "Survivor_of_Violence_or_Abuse", "WBC_OES_APPLICATION_D"."LANGUAGE" "Language35", "WBC_OES_APPLICATION_D"."LANGUAGE_OTHER" "Other_Language", "WBC_OES_APPLICATION_D"."DISABILITY_OTHER_DESC" "Other_Disability_Discription", "WBC_OES_APPLICATION_D"."STATUS_IN_CANADA_OTHER_DESC" "c38", "WBC_OES_APPLICATION_D"."ORIGIN_COUNTRY" "Country_of_Origin", "WBC_OES_APPLICATION_D"."RESUME" "Resume", "WBC_OES_APPLICATION_D"."DRENA_REQUIRED" "DRENA_Required", "WBC_OES_APPLICATION_D"."LEGAL_TO_WORK" "c42", "WBC_OES_APPLICATION_D"."CHANGE_OF_CIRCUMSATNCE" "Change_of_Circumstance", "WBC_OES_APPLICATION_D"."CHANGE_OF_CIRCUMSATNCE_REASON" "Change_of_Circumstance_Reason", "WBC_OES_APPLICATION_D"."BCEID_GUID" "BCeID_GUID", "WBC_OES_APPLICATION_D"."CLIENT_BCEID" "Client_BCeID", "WBC_OES_APPLICATION_D"."CLIENT_CONSENT" "Client_Consent", "WBC_OES_APPLICATION_D"."PROGRAM_TYPE" "Program_Type", "WBC_OES_APPLICATION_D"."REVIEWER_BCEID" "Reviewer_BCeID", "WBC_OES_APPLICATION_D"."VERSION_NUMBER" "Version_Number", "WBC_OES_APPLICATION_D"."EMPLOYMENT_FACTOR_AGE" "Age", "WBC_OES_APPLICATION_D"."EMPLOYMENT_FACTOR_LACK_HOUSING" "Lack_Housing", "WBC_OES_APPLICATION_D"."CURRENT_EMPLOYMENT_INSURANCE" "c53", "WBC_OES_APPLICATION_D"."PAST_EMPLOYMENT_INSURANCE" "c54", "WBC_OES_APPLICATION_D"."PAST_FIVE_YEARS_WORK" "c55", "WBC_OES_APPLICATION_D"."BCEA_INCOME_ASSISTANCE" "c56", "WBC_OES_APPLICATION_D"."INCOME_ASSISTANCE_DURATION" "c57", "WBC_OES_APPLICATION_D"."NON_BC_INCOME_ASSISTANCE" "c58", "WBC_OES_APPLICATION_D"."NON_BC_INCOME_ASSISTANCE_CNT" "c59", "WBC_OES_APPLICATION_D"."OTHER_FUNDING" "c60", "WBC_OES_APPLICATION_D"."OTHER_FUNDING_DESCRIPITON" "Other_Funding_Description", "WBC_OES_APPLICATION_D"."EMPLOYMENT_FACTOR_ADDICTION" "Addiction", "WBC_OES_APPLICATION_D"."VISIBLE_MINORITY" "Visible_Minority", "WBC_OES_APPLICATION_D"."CHILDREN_AT_HOME" "Child_at_Home", "WBC_OES_APPLICATION_D"."SINGLE_PARENT" "Single_Parent", "WBC_OES_APPLICATION_D"."DISABILITY_PHYSICAL" "Physical_Disability", "WBC_OES_APPLICATION_D"."DISABILITY_LEARNING" "Learning_Disablilit", "WBC_OES_APPLICATION_D"."DEVELOPMENTAL_DISABILITY" "Developmental_Disability", "WBC_OES_APPLICATION_D"."DISABILITY_VISUAL" "Visual_Disability", "WBC_OES_APPLICATION_D"."DISABILITY_HEARING" "Hearing_Disability", "WBC_OES_APPLICATION_D"."DISABILITY_MENTAL_HEALTH" "Mental_Health_Condition", "WBC_OES_APPLICATION_D"."DISABILITY_CHRONIC_ILLNESS" "Chronic_Illness", "WBC_OES_APPLICATION_D"."DISABILITY_SHORT_TERM_ILLNESS" "Short_term_Illness", "WBC_OES_APPLICATION_D"."DISABILITY_OTHER" "Other_Disability", "WBC_OES_APPLICATION_D"."EMPLOYMENT_FACTOR_LIST" "Employment_Factor_List", "WBC_OES_APPLICATION_D"."EMPLOYED" "Client_Employed_Flag", "WBC_OES_APPLICATION_D"."UNEMP_PERSONAL_SITUATION" "c77", "WBC_OES_APPLICATION_D"."UNEMP_KEEPING_JOBS" "Unemployed___Keeping_Jobs", "WBC_OES_APPLICATION_D"."UNEMP_DISABLTY_SUPPORTS_NEEDED" "c79", "WBC_OES_APPLICATION_D"."UNEMP_KIND_OF_WORK_UNKNOWN" "c80", "WBC_OES_APPLICATION_D"."UNEMP_FNCL_SUPPORTS_NEEDED" "c81", "WBC_OES_APPLICATION_D"."UNEMP_TRAINING_NEEDED" "Unemployed___Training_Needed", "WBC_OES_APPLICATION_D"."UNEMP_JOB_SEARCH_HELP_NEEDED" "c83", "WBC_OES_APPLICATION_D"."UNEMP_JOB_OFFER_LACK_RESOURCES" "Unemployed___Lack_Resources", "WBC_OES_APPLICATION_D"."UNEMP_RETURN_TO_SAME_WORK" "c85", "WBC_OES_APPLICATION_D"."UNEMP_OTHER_FACTOR" "Unemployed___Other", "WBC_OES_APPLICATION_D"."UNEMP_OTHER_FACTOR_DESC" "Unemployed___Other_Description", "WBC_OES_APPLICATION_D"."EMP_SELF_EMPLOYED" "Employed___Self_Employed", "WBC_OES_APPLICATION_D"."EMP_LOSING_JOB" "Employed___Losing_Job", "WBC_OES_APPLICATION_D"."EMP_DISABLTY_SUPPORTS_NEEDED" "Employed___Disability_Supports", "WBC_OES_APPLICATION_D"."EMP_JOB_OFFER_LACK_RESOURCES" "Employed___Lack_Resources", "WBC_OES_APPLICATION_D"."EMP_NEED_MORE_HOURS_OF_WORK" "Employed___Need_More_hours", "WBC_OES_APPLICATION_D"."EMP_TRAINING_NEEDED" "Employed___Training", "WBC_OES_APPLICATION_D"."EMP_NEW_LINE_OF_WORK" "Employed___New_Line_of_Work", "WBC_OES_APPLICATION_D"."EMP_SEEKEING_CHANGE" "Employed___Dislike_Current", "WBC_OES_APPLICATION_D"."EMP_CHALLENGING_JOB" "Employed___More_Challenge", "WBC_OES_APPLICATION_D"."EMPLOYMENT_FACTOR_OTHER" "Employed___Other", "WBC_OES_APPLICATION_D"."EMPLOYMENT_FACTOR_OTHER_DESC" "Employed___Other_Description", "WBC_OES_APPLICATION_D"."EMPLOYMENT_SITUATION_LIST" "Employment_Situation_List", "WBC_OES_APPLICATION_D"."CURRENT_EMPLOYMENT_SITUATION" "Current_Employment_Situation", "WBC_OES_APPLICATION_F"."SUBMISSION_DT" "Submission_Date", "WBC_OES_APPLICATION_F"."ICM_DT" "ICM_Date", "WBC_OES_APPLICATION_F"."DELETION_DATE" "Deletion_Date", "WBC_OES_APPLICATION_F"."DEL_STATUS_SET_BY" "Deleted_by_Code", "WBC_OES_APPLICATION_F"."APPL_ABANDONED_BY" "Application_Abandoned_By", "WBC_OES_APPLICATION_F"."SERVICE_REQ_NUM" "Service_Request_Num", "WBC_OES_APPLICATION_D"."APPLICATION_TYPE" "Application_Type", "WBC_OES_APPLICATION_D"."ACCEPTED_EMPLOYED" "Accepted_Reason___Employed", "WBC_OES_APPLICATION_D"."ACCEPTED_PRECARIOUSLY_EMPLOYED" "c109", "WBC_OES_APPLICATION_D"."ACCEPTED_SELF_EMPLOYED" "c110", "WBC_OES_APPLICATION_D"."ACCEPTED_UNEMPLOYED" "Accepted_Reason___Unemployed", "WBC_OES_APPLICATION_D"."ACCEPTED_NOT_IN_LABOUR_FORCE" "c112", "WBC_OES_APPLICATION_D"."NOTACCEPT_NOT_BC_RESIDENT" "c113", "WBC_OES_APPLICATION_D"."NOTACCEPT_NOT_UNEMPLOYED" "c114", "WBC_OES_APPLICATION_D"."NOTACCEPT_NOT_ELIBIBLE" "c115", "WBC_OES_APPLICATION_D"."NOTACCEPT_900_SIN" "c116", "WBC_OES_APPLICATION_D"."NOTACCEPT_FULL_TIME_STUDENT" "c117", "WBC_OES_APPLICATION_D"."NOTACCEPT_DRUG_ALCOHOL_ABUSE" "c118", "WBC_OES_APPLICATION_D"."NOTACCEPT_HEALTH" "c119", "WBC_OES_APPLICATION_D"."NOTACCEPT_OTHER" "Not_Accepted_Reason___Other", "WBC_OES_APPLICATION_D"."NOT_ACCEPTED_REASONS_LIST" "Not_Accepted_Reason_Lsit", "WBC_OES_APPLICATION_D"."INTERVIEW_METHOD_IN_PERSON" "Interview_In_Person", "WBC_OES_APPLICATION_D"."INTERVIEW_METHOD_PHONE" "Interview_By_Telephone", "WBC_OES_APPLICATION_D"."INTERVIEW_METHOD_VIDEO_CALL" "Interview_By_Video_Call", "WBC_OES_APPLICATION_D"."INTERVIEW_METHOD_WITH_ATTNDENT" "Interview_With_Attendant", "WBC_OES_APPLICATION_D"."INTERVIEW_METHOD_OTHER" "Interview_Other", "WBC_OES_APPLICATION_D"."EMPLOYMENT_GOAL" "Employment_Goal", "WBC_OES_APPLICATION_D"."HOURS_PER_WEEK" "Hours_per_Week_Goal", "WBC_OES_APPLICATION_D"."WAGE_PER_HOUR" "Wage_Per_Hour_Goal", "WBC_OES_APPLICATION_D"."EMPLOYMENT_GOAL_SEASONAL" "Goal_is_Seasonal", "WBC_OES_APPLICATION_D"."EMPLOYMENT_GOAL_CASUAL" "Goal_is_Casual", "WBC_OES_APPLICATION_D"."EMPLOYMENT_GOAL_RELOCATE" "Can_Relocate", "WBC_OES_APPLICATION_D"."EMPLOYMENT_GOAL_TRANSPORT" "Has_Transportation", "WBC_OES_APPLICATION_D"."PORTAL_APPLICATION_NUMBER" "OES_Application_Num", "WBC_OES_APPLICATION_D"."OFFICE" "Office", "WBC_OES_APPLICATION_D"."CATCHMENT" "Catchment_Selected", "WBC_OES_APPLICATION_D"."SERVICE_SELECTED" "Service_Selected", "WBC_OES_APPLICATION_D"."ACCEPTED_INTO_CASE_MGMT" "Accepted_into_Case_Management", "WBC_OES_APPLICATION_D"."NOTACCEPT_CASE_MGMT_OTHER_DESC" "c139", "WBC_OES_APPLICATION_D"."EXPLORE_WITH_CLIENT" "Explore_with_Client"
 from (
select "WBC_OES_APPLICATION_F"."TAAAPA_WID" "TAAAPA_WID", "WBC_OES_APPLICATION_F"."SUBMISSION_DT" "SUBMISSION_DT", "WBC_OES_APPLICATION_F"."ICM_DT" "ICM_DT", "WBC_OES_APPLICATION_F"."APPLICATION_TYPE_WID" "APPLICATION_TYPE_WID", "WBC_OES_APPLICATION_F"."SERVICE_REQ_NUM" "SERVICE_REQ_NUM", "WBC_OES_APPLICATION_F"."DELETION_DATE" "DELETION_DATE", "WBC_OES_APPLICATION_F"."DEL_STATUS_SET_BY" "DEL_STATUS_SET_BY", "WBC_OES_APPLICATION_F"."APPL_ABANDONED_BY" "APPL_ABANDONED_BY"
 from "CDW"."WBC_OES_APPLICATION_F" "WBC_OES_APPLICATION_F"
 ) "WBC_OES_APPLICATION_F" LEFT OUTER JOIN "CDW"."WBC_OES_APPLICATION_D" "WBC_OES_APPLICATION_D" on "WBC_OES_APPLICATION_F"."TAAAPA_WID"="WBC_OES_APPLICATION_D"."TAAAPA_WID"
 where "WBC_OES_APPLICATION_F"."APPLICATION_TYPE_WID" in (5, 6)


"""
data = pd.read_sql(q, con=conn)
print(data.columns)

Index(['Explaining_to_Others', 'Getting_Along_with_Others', 'New_Things',
       'Organizing', 'Problem_Solving', 'Reading_and_Understanding',
       'Using_Computers', 'Working_with_Documents', 'Working_with_Math',
       'Writing',
       ...
       'Goal_is_Casual', 'Can_Relocate', 'Has_Transportation',
       'OES_Application_Num', 'Office', 'Catchment_Selected',
       'Service_Selected', 'Accepted_into_Case_Management', 'c139',
       'Explore_with_Client'],
      dtype='object', length=140)


In [6]:
link = """
select distinct "WBC_CASE_D"."CASE_NUM" "Case_Number", "WBC_CATCHMENT_AREA_D"."REGION_CODE" "Region_Code", "WBC_CATCHMENT_AREA_D"."CATCHMENT" "Catchment", "WBC_OES_APPLICATION_D"."EXPLAINING_TO_OTHERS" "Explaining_to_Others", "WBC_OES_APPLICATION_D"."GET_ALONG_WITH_OTHERS" "Getting_Along_with_Others", "WBC_OES_APPLICATION_D"."LEARNING_NEW_THINGS" "New_Things", "WBC_OES_APPLICATION_D"."ORGANIZING_INFORMATION" "Organizing", "WBC_OES_APPLICATION_D"."PROBLEM_SOLVING" "Problem_Solving", "WBC_OES_APPLICATION_D"."READING_INFORMATION" "Reading_and_Understanding", "WBC_OES_APPLICATION_D"."USING_COMPUTERS" "Using_Computers", "WBC_OES_APPLICATION_D"."CREATING_DOCUMENTS" "Working_with_Documents", "WBC_OES_APPLICATION_D"."WORKING_WITH_MATH" "Working_with_Math", "WBC_OES_APPLICATION_D"."WRITING_INFORMATION" "Writing", "WBC_OES_APPLICATION_D"."FRANCOPHONE" "Francophone", "WBC_OES_APPLICATION_D"."RECENT_IMMIGRANT" "Recent_Immigrant", "WBC_OES_APPLICATION_D"."INDIGENOUS_PERSON" "Indigenous_Person", "WBC_OES_APPLICATION_D"."PERSON_WITH_DISABILITY" "Person_with_a_Disability", "WBC_OES_APPLICATION_D"."SURVIVOR_OF_VIOLENCE" "Survivor_of_Violence_or_Abuse", "WBC_OES_APPLICATION_D"."LANGUAGE" "Language19", "WBC_OES_APPLICATION_D"."LANGUAGE_OTHER" "Other_Language", "WBC_OES_APPLICATION_D"."DISABILITY_OTHER_DESC" "Other_Disability_Discription", "WBC_OES_APPLICATION_D"."STATUS_IN_CANADA_OTHER_DESC" "c22", "WBC_OES_APPLICATION_D"."SCHOOL_LEVEL" "Education_Level", "WBC_OES_APPLICATION_D"."CURRENTLY_IN_SCHOOL" "Currently_In_School", "WBC_OES_APPLICATION_D"."CURRENTLY_IN_SCHOOL_STATUS" "c25", "WBC_OES_APPLICATION_D"."AREA_OF_STUDY" "Education_Area_of_Study", "WBC_OES_APPLICATION_D"."INDIGENOUS_TYPE" "Indigenous_Type", "WBC_OES_APPLICATION_D"."STATUS_INDIAN" "Status_Indian", "WBC_OES_APPLICATION_D"."DRENA_REQUIRED" "DRENA_Required", "WBC_OES_APPLICATION_D"."LEGAL_TO_WORK" "c30", "WBC_OES_APPLICATION_D"."CHANGE_OF_CIRCUMSATNCE" "Change_of_Circumstance", "WBC_OES_APPLICATION_D"."CHANGE_OF_CIRCUMSATNCE_REASON" "Change_of_Circumstance_Reason", "WBC_OES_APPLICATION_D"."BCEID_GUID" "BCeID_GUID", "WBC_OES_APPLICATION_D"."CLIENT_BCEID" "Client_BCeID", "WBC_OES_APPLICATION_D"."CLIENT_CONSENT" "Client_Consent", "WBC_OES_APPLICATION_D"."PROGRAM_TYPE" "Program_Type", "WBC_OES_APPLICATION_D"."CURRENT_EMPLOYMENT_INSURANCE" "c37", "WBC_OES_APPLICATION_D"."PAST_EMPLOYMENT_INSURANCE" "c38", "WBC_OES_APPLICATION_D"."PAST_FIVE_YEARS_WORK" "c39", "WBC_OES_APPLICATION_D"."BCEA_INCOME_ASSISTANCE" "c40", "WBC_OES_APPLICATION_D"."INCOME_ASSISTANCE_DURATION" "c41", "WBC_OES_APPLICATION_D"."NON_BC_INCOME_ASSISTANCE" "c42", "WBC_OES_APPLICATION_D"."NON_BC_INCOME_ASSISTANCE_CNT" "c43", "WBC_OES_APPLICATION_D"."OTHER_FUNDING" "c44", "WBC_OES_APPLICATION_D"."EMPLOYMENT_FACTOR_AGE" "Age", "WBC_OES_APPLICATION_D"."EMPLOYMENT_FACTOR_LACK_HOUSING" "Lack_Housing", "WBC_OES_APPLICATION_D"."EMPLOYMENT_FACTOR_ADDICTION" "Addiction", "WBC_OES_APPLICATION_D"."VISIBLE_MINORITY" "Visible_Minority", "WBC_OES_APPLICATION_D"."CHILDREN_AT_HOME" "Child_at_Home", "WBC_OES_APPLICATION_D"."SINGLE_PARENT" "Single_Parent", "WBC_OES_APPLICATION_D"."DISABILITY_PHYSICAL" "Physical_Disability", "WBC_OES_APPLICATION_D"."DISABILITY_LEARNING" "Learning_Disablilit", "WBC_OES_APPLICATION_D"."DEVELOPMENTAL_DISABILITY" "Developmental_Disability", "WBC_OES_APPLICATION_D"."DISABILITY_VISUAL" "Visual_Disability", "WBC_OES_APPLICATION_D"."DISABILITY_HEARING" "Hearing_Disability", "WBC_OES_APPLICATION_D"."DISABILITY_MENTAL_HEALTH" "Mental_Health_Condition", "WBC_OES_APPLICATION_D"."DISABILITY_CHRONIC_ILLNESS" "Chronic_Illness", "WBC_OES_APPLICATION_D"."DISABILITY_SHORT_TERM_ILLNESS" "Short_term_Illness", "WBC_OES_APPLICATION_D"."DISABILITY_OTHER" "Other_Disability", "WBC_OES_APPLICATION_D"."INTERVIEW_METHOD_IN_PERSON" "Interview_In_Person", "WBC_OES_APPLICATION_D"."EMPLOYED" "Client_Employed_Flag", "WBC_OES_APPLICATION_D"."UNEMP_PERSONAL_SITUATION" "c62", "WBC_OES_APPLICATION_D"."UNEMP_KEEPING_JOBS" "Unemployed___Keeping_Jobs", "WBC_OES_APPLICATION_D"."UNEMP_DISABLTY_SUPPORTS_NEEDED" "c64", "WBC_OES_APPLICATION_D"."UNEMP_KIND_OF_WORK_UNKNOWN" "c65", "WBC_OES_APPLICATION_D"."UNEMP_FNCL_SUPPORTS_NEEDED" "c66", "WBC_OES_APPLICATION_D"."UNEMP_TRAINING_NEEDED" "Unemployed___Training_Needed", "WBC_OES_APPLICATION_D"."UNEMP_JOB_SEARCH_HELP_NEEDED" "c68", "WBC_OES_APPLICATION_D"."UNEMP_JOB_OFFER_LACK_RESOURCES" "Unemployed___Lack_Resources", "WBC_OES_APPLICATION_D"."UNEMP_RETURN_TO_SAME_WORK" "c70", "WBC_OES_APPLICATION_D"."UNEMP_OTHER_FACTOR" "Unemployed___Other", "WBC_OES_APPLICATION_D"."UNEMP_OTHER_FACTOR_DESC" "Unemployed___Other_Description", "WBC_OES_APPLICATION_D"."EMP_SELF_EMPLOYED" "Employed___Self_Employed", "WBC_OES_APPLICATION_D"."EMP_LOSING_JOB" "Employed___Losing_Job", "WBC_OES_APPLICATION_D"."EMP_DISABLTY_SUPPORTS_NEEDED" "Employed___Disability_Supports", "WBC_OES_APPLICATION_D"."EMP_JOB_OFFER_LACK_RESOURCES" "Employed___Lack_Resources", "WBC_OES_APPLICATION_D"."EMP_NEED_MORE_HOURS_OF_WORK" "Employed___Need_More_hours", "WBC_OES_APPLICATION_D"."EMP_TRAINING_NEEDED" "Employed___Training", "WBC_OES_APPLICATION_D"."EMP_NEW_LINE_OF_WORK" "Employed___New_Line_of_Work", "WBC_OES_APPLICATION_D"."EMP_SEEKEING_CHANGE" "Employed___Dislike_Current", "WBC_OES_APPLICATION_D"."EMP_CHALLENGING_JOB" "Employed___More_Challenge", "WBC_OES_APPLICATION_D"."EMPLOYMENT_FACTOR_OTHER" "Employed___Other", "WBC_OES_APPLICATION_D"."EMPLOYMENT_FACTOR_OTHER_DESC" "Employed___Other_Description", "WBC_OES_APPLICATION_D"."EMPLOYMENT_SITUATION_LIST" "Employment_Situation_List", "WBC_OES_APPLICATION_D"."INTERVIEW_METHOD_PHONE" "Interview_By_Telephone", "WBC_OES_APPLICATION_D"."INTERVIEW_METHOD_VIDEO_CALL" "Interview_By_Video_Call", "WBC_OES_APPLICATION_D"."INTERVIEW_METHOD_WITH_ATTNDENT" "Interview_With_Attendant", "WBC_OES_APPLICATION_D"."INTERVIEW_METHOD_OTHER" "Interview_Other", "WBC_OES_APPLICATION_D"."EMPLOYMENT_GOAL" "Employment_Goal", "WBC_OES_APPLICATION_D"."HOURS_PER_WEEK" "Hours_per_Week_Goal", "WBC_OES_APPLICATION_D"."WAGE_PER_HOUR" "Wage_Per_Hour_Goal", "WBC_OES_APPLICATION_D"."EMPLOYMENT_GOAL_SEASONAL" "Goal_is_Seasonal", "WBC_OES_APPLICATION_D"."EMPLOYMENT_GOAL_CASUAL" "Goal_is_Casual", "WBC_OES_APPLICATION_D"."EMPLOYMENT_GOAL_RELOCATE" "Can_Relocate", "WBC_OES_APPLICATION_D"."EMPLOYMENT_GOAL_TRANSPORT" "Has_Transportation", "WBC_OES_APPLICATION_D"."PORTAL_APPLICATION_NUMBER" "OES_Application_Num", "WBC_OES_APPLICATION_D"."OFFICE" "Office", "WBC_OES_APPLICATION_D"."CATCHMENT" "Catchment_Selected", "WBC_OES_APPLICATION_D"."SERVICE_SELECTED" "Service_Selected", "WBC_OES_APPLICATION_D"."ACCEPTED_INTO_CASE_MGMT" "Accepted_into_Case_Management", "WBC_OES_APPLICATION_D"."NOTACCEPT_CASE_MGMT_OTHER_DESC" "c101", "WBC_OES_APPLICATION_D"."EXPLORE_WITH_CLIENT" "Explore_with_Client"
 from ((("CDW"."WBC_OFFICE_D" "WBC_OFFICE_D" INNER JOIN (
select "WBC_CATCHMENT_AREA_D"."CATCHMENT_AREA_WID" "CATCHMENT_AREA_WID", "WBC_CATCHMENT_AREA_D"."CATCHMENT" "CATCHMENT", "WBC_CATCHMENT_AREA_D"."REGION_CODE" "REGION_CODE"
 from "CDW"."WBC_CATCHMENT_AREA_D" "WBC_CATCHMENT_AREA_D"
 ) "WBC_CATCHMENT_AREA_D" on "WBC_OFFICE_D"."CATCHMENT_AREA_WID"="WBC_CATCHMENT_AREA_D"."CATCHMENT_AREA_WID") INNER JOIN (
select "WBC_OES_APPLICATION_F"."TAAAPA_WID" "TAAAPA_WID", "WBC_OES_APPLICATION_F"."OFFICE_ORG_WID" "OFFICE_ORG_WID", "WBC_OES_APPLICATION_F"."CASE_WID" "CASE_WID"
 from "CDW"."WBC_OES_APPLICATION_F" "WBC_OES_APPLICATION_F"
) "WBC_OES_APPLICATION_F" on "WBC_OES_APPLICATION_F"."OFFICE_ORG_WID"="WBC_OFFICE_D"."OFFICE_ORG_WID") INNER JOIN "CDW"."WBC_CASE_D" "WBC_CASE_D" on "WBC_OES_APPLICATION_F"."CASE_WID"="WBC_CASE_D"."CASE_WID") LEFT OUTER JOIN "CDW"."WBC_OES_APPLICATION_D" "WBC_OES_APPLICATION_D" on "WBC_OES_APPLICATION_F"."TAAAPA_WID"="WBC_OES_APPLICATION_D"."TAAAPA_WID"



"""
link = pd.read_sql(link, con=conn)
print(link.head)


<bound method NDFrame.head of             Case_Number Region_Code                      Catchment  \
0         1-56052045193        EDR1          05-ES COWICHAN VALLEY   
1                     0        EDR2               17-ES TRI-CITIES   
2       1-421181-191989        EDR3        30-ES THOMPSON - NICOLA   
3         1-56177037887        EDR2                  24-ES BURNABY   
4         1-57127199186        EDR2          14-ES VANCOUVER SOUTH   
...                 ...         ...                            ...   
123207  1-421193-429015         UNK       Source Value was Invalid   
123208    1-63799368650        EDR1  ASSISTIVE TECHNOLOGY SERVICES   
123209    1-58288244815        EDR2            APPRENTICE SERVICES   
123210  1-421189-197001        EDR2            APPRENTICE SERVICES   
123211    1-59026100477        EDR2            APPRENTICE SERVICES   

       Explaining_to_Others Getting_Along_with_Others             New_Things  \
0         I am good at this         I am good at 

In [45]:
link2 = link[link['Case_Number'].isin(df_combo['Case_Number'])]
print(link2.shape)
data = link2
#data.head().to_csv('sql_data.csv')

(52277, 102)


In [46]:
l = []
for col in data.columns:
    #print("--------COLUMN--------")
    if data[col].nunique() > 100:
        print(col)
        
        if (col != 'Case_Number') & (col != 'Wage_Per_Hour_Goal'):
            l.append(col)

    #print(data[col].nunique())
data = data.drop(l, axis = 1)
print(data.dtypes)
for x in data.columns:
    print(x)
    print(data[x].nunique())

Case_Number
Other_Language
Other_Disability_Discription
c22
Education_Area_of_Study
Change_of_Circumstance_Reason
BCeID_GUID
Client_BCeID
Unemployed___Other_Description
Employed___Other_Description
Employment_Situation_List
Employment_Goal
Wage_Per_Hour_Goal
OES_Application_Num
Office
Explore_with_Client
Case_Number                      object
Region_Code                      object
Catchment                        object
Explaining_to_Others             object
Getting_Along_with_Others        object
                                  ...  
Has_Transportation               object
Catchment_Selected               object
Service_Selected                 object
Accepted_into_Case_Management    object
c101                             object
Length: 88, dtype: object
Case_Number
50397
Region_Code
7
Catchment
45
Explaining_to_Others
2
Getting_Along_with_Others
2
New_Things
2
Organizing
2
Problem_Solving
2
Reading_and_Understanding
2
Using_Computers
2
Working_with_Documents
2
Working_with_Math

In [111]:
print(data['Case_Number'].dtypes)
pp = df_combo[['Case_Number', 'sus']]
print(pp.dtypes)
tt=data.merge(df_combo[['Case_Number', 'sus']], on = 'Case_Number', how = 'left')
print(tt.shape)
tt=tt.drop('Case_Number', axis = 1)

object
Case_Number    object
sus             int64
dtype: object
(52277, 89)


In [27]:
ref = pd.read_csv('sql_data.csv')
print(ref.coding.value_counts())
ref[ref['var_name'] == "Interview_By_Telephone"].coding

binary     44
ordinal    31
no         16
one hot     8
float       2
label       1
Name: coding, dtype: int64


84    binary
Name: coding, dtype: object

In [115]:
from sklearn import preprocessing


tst = tt
print(tst.dtypes)
le = preprocessing.LabelEncoder()
oh = preprocessing.OneHotEncoder()
bi = preprocessing.OneHotEncoder()
ord = preprocessing.OrdinalEncoder()


for col in tst.columns:
    print(col)
    #print(tst[col][0:10])
    #print(ref[ref['var_name'] == col].coding)
    if col == 'sus':
        tst[col] =tst[col]
    elif ref[ref['var_name'] == col].coding.item() == 'binary':
        X=tst[col].astype(str).values.reshape(-1,1)
        tst = tst.drop(col, axis = 1)
        tst = pd.concat([tst, pd.DataFrame(bi.fit_transform(X).toarray())], axis=1)
    elif ref[ref['var_name'] == col].coding.item() == 'ordinal':
        X=tst[col].astype(str).values.reshape(-1,1)
        tst = tst.drop(col, axis = 1)
        tst = pd.concat([tst, pd.DataFrame(ord.fit_transform(X))], axis=1)
        print(pd.DataFrame(ord.fit_transform(X)))
    elif ref[ref['var_name'] == col].coding.item() == 'one hot':
        X=tst[col].astype(str).values.reshape(-1,1)
        tst = tst.drop(col, axis = 1)
        tst = pd.concat([tst, pd.DataFrame(oh.fit_transform(X).toarray())], axis=1)
    elif ref[ref['var_name'] == col].coding.item() == 'float':
        tst[col] =tst[col]
    elif ref[ref['var_name'] == col].coding.item() == 'no':
        tst = tst.drop(col, axis = 1)

tst.head()

Region_Code                      object
Catchment                        object
Explaining_to_Others             object
Getting_Along_with_Others        object
New_Things                       object
                                  ...  
Catchment_Selected               object
Service_Selected                 object
Accepted_into_Case_Management    object
c101                             object
sus                               int64
Length: 88, dtype: object
Region_Code
Catchment
Explaining_to_Others
         0
0      0.0
1      0.0
2      0.0
3      1.0
4      0.0
...    ...
52272  0.0
52273  0.0
52274  0.0
52275  0.0
52276  0.0

[52277 rows x 1 columns]
Getting_Along_with_Others
         0
0      0.0
1      0.0
2      0.0
3      0.0
4      0.0
...    ...
52272  0.0
52273  0.0
52274  0.0
52275  0.0
52276  0.0

[52277 rows x 1 columns]
New_Things
         0
0      0.0
1      0.0
2      0.0
3      0.0
4      0.0
...    ...
52272  0.0
52273  0.0
52274  0.0
52275  0.0
52276  1.0

[5227

,Hours_per_Week_Goal,Wage_Per_Hour_Goal,c101,sus,0,1,2,3,4,5,...,1,2,0,1,2,0,1,2,0,1
0,35.0,14.0,None,1,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
1,40.0,20.0,None,1,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
2,35.0,25.0,None,1,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
3,40.0,NaN,None,1,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
4,40.0,25.0,None,1,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


In [116]:

tst.to_csv('preprocessed_data_02_11_21.csv')
#print(tst.c101.value_counts())
tst.head().to_csv('preprocessed_head.csv')
#tst = tst.drop('c101', axis = 1)
features = tst.drop('sus', axis = 1)
label = tst['sus']
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(features, label)
from sklearn.preprocessing import StandardScaler

x_train = x_train.fillna(value=0)
x_test = x_test.fillna(value=0)
scaler = StandardScaler()
scaler.fit(x_train)

x_train = scaler.transform(x_train)
print(x_train[:2])
x_test = scaler.transform(x_test)
model = KNeighborsClassifier(n_neighbors=3)

# Train the model using the training sets
model.fit(x_train,y_train)

[[ 0.68120489  0.63109898  0.         -0.43327934  0.97230173 -0.42502852
  -0.23367425 -0.27801612 -0.15336597 -0.17093039 -0.13344837 -0.12167674
  -0.14706304 -0.16556021 -0.13531717 -0.13936562 -0.18963525 -0.08010825
  -0.09394492 -0.16053192 -0.17730607 -0.19861927 -0.13745156 -0.13716232
  -0.13580499 -0.11097763 -0.21872178 -0.13275371 -0.2853476  -0.10317934
  -0.17951307 -0.10241919  8.18948767 -0.22417212 -0.16417276 -0.0888371
  -0.14823537 -0.1930728  -0.1670987  -0.1021646  -0.18963525 -0.13822013
  -0.22803579 -0.1277927  -0.13295253 -0.14787556 -0.1148012  -0.15587359
  -0.05767812 -0.08526769 -0.08677225 -0.0706998  -0.21601933 -0.14078538
  -0.09504396 -0.45805098 -0.29665734 -0.37462832 -0.49456173 -0.42842352
  -0.38609653  1.43839773  1.08293684  1.17247522 -0.56782832 -0.09826995
   0.21256891 -0.05632704 -0.17738257 -0.13492574 -1.51629209 -0.05632704
   1.59538624 -0.12933024  0.37704396 -0.05632704 -0.34225116 -0.19257477
   0.68313188 -0.05632704 -0.62246727 -

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=3, p=2,
                     weights='uniform')

In [117]:

y_pred = model.predict(x_test)
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[3538 3164]
 [2511 3857]]
              precision    recall  f1-score   support

           0       0.58      0.53      0.55      6702
           1       0.55      0.61      0.58      6368

    accuracy                           0.57     13070
   macro avg       0.57      0.57      0.57     13070
weighted avg       0.57      0.57      0.57     13070



In [120]:
from sklearn import datasets, ensemble
from sklearn.inspection import permutation_importance
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
params = {'n_estimators': 600,
          'max_depth': 12,
          'min_samples_split': 5,
          'learning_rate': 0.01,
          'loss': 'ls'}

reg = ensemble.GradientBoostingRegressor(**params)
reg.fit(x_train, y_train)

mse = mean_squared_error(y_test, reg.predict(x_test))
print("The mean squared error (MSE) on test set: {:.4f}".format(mse))

y_pred = reg.predict(x_test)



The mean squared error (MSE) on test set: 0.2310


In [121]:
print(classification_report(y_test, y_pred.round()))
print(confusion_matrix(y_test, y_pred.round()))

              precision    recall  f1-score   support

           0       0.63      0.61      0.62      6702
           1       0.60      0.62      0.61      6368

    accuracy                           0.62     13070
   macro avg       0.62      0.62      0.62     13070
weighted avg       0.62      0.62      0.62     13070

[[4107 2595]
 [2428 3940]]


In [125]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(solver='lbfgs', alpha=1e-4,hidden_layer_sizes=(15, 12), random_state=2, max_iter = 8000)
clf.fit(x_train, y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(15, 12), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=8000,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=2, shuffle=True, solver='lbfgs',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [126]:
y_pred = clf.predict(x_test)
print(classification_report(y_test, y_pred.round()))
print(confusion_matrix(y_test, y_pred.round()))

              precision    recall  f1-score   support

           0       0.61      0.62      0.61      6702
           1       0.59      0.58      0.59      6368

    accuracy                           0.60     13070
   macro avg       0.60      0.60      0.60     13070
weighted avg       0.60      0.60      0.60     13070

[[4144 2558]
 [2659 3709]]


In [127]:
from sklearn.decomposition import PCA
# Make an instance of the Model
pca = PCA(.70)
pca.fit(x_train)
print(pca.n_components_)
x_train_pca = pca.transform(x_train)
x_test_pca = pca.transform(x_test)

54


In [128]:
model = KNeighborsClassifier(n_neighbors=3)

# Train the model using the training sets
model.fit(x_train_pca,y_train)
y_pred = model.predict(x_test_pca)
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[3677 3025]
 [2720 3648]]
              precision    recall  f1-score   support

           0       0.57      0.55      0.56      6702
           1       0.55      0.57      0.56      6368

    accuracy                           0.56     13070
   macro avg       0.56      0.56      0.56     13070
weighted avg       0.56      0.56      0.56     13070



In [130]:
params = {'n_estimators': 200,
          'max_depth': 15,
          'min_samples_split': 5,
          'learning_rate': 0.01,
          'loss': 'ls'}
reg = ensemble.GradientBoostingRegressor(**params)
reg.fit(x_train_pca, y_train)

mse = mean_squared_error(y_test, reg.predict(x_test_pca))
print("The mean squared error (MSE) on test set: {:.4f}".format(mse))

y_pred = reg.predict(x_test_pca)

print(classification_report(y_test, y_pred.round()))
print(confusion_matrix(y_test, y_pred.round()))

The mean squared error (MSE) on test set: 0.2365
              precision    recall  f1-score   support

           0       0.61      0.62      0.61      6702
           1       0.59      0.58      0.59      6368

    accuracy                           0.60     13070
   macro avg       0.60      0.60      0.60     13070
weighted avg       0.60      0.60      0.60     13070

[[4136 2566]
 [2650 3718]]


In [96]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(solver='adam', alpha=1e-4,hidden_layer_sizes=(10, 10), random_state=1, max_iter = 2000)
clf.fit(x_train_pca, y_train)
y_pred = clf.predict(x_test_pca)
print(classification_report(y_test, y_pred.round()))
print(confusion_matrix(y_test, y_pred.round()))

              precision    recall  f1-score   support

           0       0.61      0.56      0.58      6708
           1       0.57      0.62      0.60      6362

    accuracy                           0.59     13070
   macro avg       0.59      0.59      0.59     13070
weighted avg       0.59      0.59      0.59     13070

[[3740 2968]
 [2408 3954]]


In [140]:
from sklearn import preprocessing
# tt=data.merge(df_combo[['Case_Number', 'sus']], on = 'Case_Number', how = 'left')
# tt.drop('Case_Number', axis = 1)
# print(tt.shape)

# tst = tt
# tst.fillna(0, inplace=True)
# print(tst.columns)
# features = tst.drop('sus', axis = 1)
# label = tst['sus']
# features = features.drop('Case_Number', axis = 1)
# #features = features.drop('c22', axis = 1)
# features = features.drop('Wage_Per_Hour_Goal', axis = 1)
# for col in features.columns:
#     print(col)
#     print(features[col].nunique())
# for col in features.columns:
#     features[col]=features[col].astype(str)
# #print(tst.dtypes)
# le = preprocessing.OneHotEncoder()

# le.fit(features)
# tst = pd.DataFrame(le.transform(features).toarray())
# print(tst.head)
# print(tst[tst.eq('EDR2').any(1)])
# features = tst.drop('sus', axis = 1)
# label = tst['sus']
tst=pd.read_csv('preprocessed_data_02_11_21.csv')
tst=tst.fillna(value=0)
print(tst.head())
features = tst.drop('sus', axis = 1)
label = tst['sus']
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(features, label)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
print(x_train.head())
scaler.fit(x_train)

x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)

from sklearn.decomposition import PCA
# Make an instance of the Model
pca = PCA(.90)
pca.fit(x_train)
print(pca.n_components_)
x_train_pca = pca.transform(x_train)
x_test_pca = pca.transform(x_test)


   Unnamed: 0  Hours_per_Week_Goal  Wage_Per_Hour_Goal  c101  sus    0    1  \
0           0                 35.0                14.0   0.0    1  1.0  0.0   
1           1                 40.0                20.0   0.0    1  0.0  1.0   
2           2                 35.0                25.0   0.0    1  0.0  1.0   
3           3                 40.0                 0.0   0.0    1  0.0  0.0   
4           4                 40.0                25.0   0.0    1  0.0  0.0   

     2    3    4  ...  1.42  2.42  0.74  1.43  2.43  0.75  1.44  2.44  0.76  \
0  0.0  0.0  0.0  ...   0.0   0.0   1.0   0.0   0.0   0.0   0.0   1.0   0.0   
1  0.0  0.0  0.0  ...   0.0   0.0   1.0   0.0   0.0   0.0   0.0   1.0   0.0   
2  0.0  0.0  0.0  ...   0.0   0.0   1.0   0.0   0.0   1.0   0.0   0.0   0.0   
3  1.0  0.0  0.0  ...   0.0   1.0   0.0   0.0   1.0   1.0   0.0   0.0   0.0   
4  0.0  1.0  0.0  ...   0.0   1.0   1.0   0.0   0.0   1.0   0.0   0.0   0.0   

   1.45  
0   1.0  
1   1.0  
2   1.0  
3   1.0  


In [141]:
model = KNeighborsClassifier(n_neighbors=9)

# Train the model using the training sets
model.fit(x_train_pca,y_train)
y_pred = model.predict(x_test_pca)
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[3670 3075]
 [2274 4051]]
              precision    recall  f1-score   support

           0       0.62      0.54      0.58      6745
           1       0.57      0.64      0.60      6325

    accuracy                           0.59     13070
   macro avg       0.59      0.59      0.59     13070
weighted avg       0.59      0.59      0.59     13070



In [142]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(solver='adam', alpha=1e-4,hidden_layer_sizes=(10, 10), random_state=1, max_iter = 2000)
clf.fit(x_train_pca, y_train)
y_pred = clf.predict(x_test_pca)
print(classification_report(y_test, y_pred.round()))
print(confusion_matrix(y_test, y_pred.round()))

              precision    recall  f1-score   support

           0       0.62      0.65      0.63      6745
           1       0.60      0.57      0.59      6325

    accuracy                           0.61     13070
   macro avg       0.61      0.61      0.61     13070
weighted avg       0.61      0.61      0.61     13070

[[4376 2369]
 [2715 3610]]


In [143]:
reg = ensemble.GradientBoostingRegressor(**params)
reg.fit(x_train_pca, y_train)

mse = mean_squared_error(y_test, reg.predict(x_test_pca))
print("The mean squared error (MSE) on test set: {:.4f}".format(mse))

y_pred = reg.predict(x_test_pca)

print(classification_report(y_test, y_pred.round()))
print(confusion_matrix(y_test, y_pred.round()))

The mean squared error (MSE) on test set: 0.2344
              precision    recall  f1-score   support

           0       0.62      0.62      0.62      6745
           1       0.59      0.59      0.59      6325

    accuracy                           0.60     13070
   macro avg       0.60      0.60      0.60     13070
weighted avg       0.60      0.60      0.60     13070

[[4170 2575]
 [2590 3735]]


In [144]:
print(tt['Explore_with_Client'].value_counts())

KeyError: 'Explore_with_Client'